In [1]:
from iot.core import get_bitalino_df, get_glasses_df, get_tablet_df, build_graphs

In [3]:
output_path = 'C:/Users/artem/PycharmProjects/iot/output'
sync_names = ['glasses', 'tablet']
base_name  = 'bitalino'
experiment_file = 'experiments/exp10.txt'
experiment_name = experiment_file.split('/')[-1].split('.')[0]

In [4]:
bitalino_path, glasses_path, tablet_path = lines

In [5]:
dic_df = {}
if bitalino_path:
    dic_df['bitalino'], dic_bitalino = get_bitalino_df(bitalino_path, N = 100000)
dic_df['glasses'], dic_glasses = get_glasses_df(glasses_path, N = 10000)
if tablet_path:
    dic_df['tablet'], dic_tablet = get_tablet_df(tablet_path, N = 10000)

In [6]:
build_graphs(dic_df, base_name, sync_names, experiment_name, output_path)